In [ ]:
import pandas as pd
from openai import OpenAI
import tiktoken
import time
import os

client = OpenAI(
    api_key=os.getenv(""), 
    base_url=""
)

# 最大 token 限制
MAX_TOKENS = 4000  # 将 token 限制改为 4000

# 使用 tiktoken 进行 token 数量估算
def count_tokens(text):
    """估算文本的 token 数"""
    enc = tiktoken.get_encoding("cl100k_base")  # 获取与 GPT-3.5 和 GPT-4 兼容的 tokenizer
    tokens = enc.encode(text)
    return len(tokens)

# 发送请求并处理 token 限制的 prompt 函数
def prompt(s, history,llm_name=""):
    """发送请求，确保 token 数在模型限制范围内"""
    # 将 s0 作为系统角色消息添加到历史记录中（如果尚未添加）
    if not history[0]["content"]:
        history[0]["content"] = s0
    
    # 将用户输入添加到历史记录中
    context = history + [{"role": "user", "content": s}]
    
    # 计算当前 context 的 token 总数
    total_tokens = sum(count_tokens(item['content']) for item in context)
    print(f"当前总 token 数：{total_tokens}")

    # 如果 token 数量超过限制，裁剪历史记录
    while total_tokens > MAX_TOKENS:
        context.pop(1)  # 移除最早的非系统消息（s0 已保留）
        total_tokens = sum(count_tokens(item['content']) for item in context)
        print(f"裁剪后总 token 数：{total_tokens}")

    try:
        # 调用千帆接口
        response = client.chat.completions.create(
            model=llm_name,  # 使用选择的模型
            messages=context  # 传递消息
        )
        
        # 获取大模型的反馈内容
        feedback = response.choices[0].message.content.strip('"')
        print(feedback)
        
        return context + [{"role": "assistant", "content": feedback}]
    
    except Exception as e:
        print(f"Error: {e}")
        print("等待 60 秒后重试...")
        time.sleep(60)  # 等待后重试
        return prompt(s, history,llm_name)  # 重试请求


# 初始化第一个提示信息
s0 = """在图书直播的场景下，社会支持理论中的情感支持、信息支持和有形支持可以通过以下方式体现：

情感支持
情感支持主要指的是提供关怀、鼓励和安慰，以满足个体的情感需求。在图书直播中，情感支持可能包括：
主播对观众的问候和欢迎，如“欢迎橙子、莹宝我们的微笑，欢迎金耳，欢迎青鸟，欢迎来到我们直播间。”
主播对观众的积极反馈和感谢，如“大家晚上好，这个声音能听见吗？晚上好诶，我这边可以听见，是吧？好的好的。”
主播与观众之间的互动，创造亲切感，如“感觉忙碌的一周又过了一半，可以，是吧？”
主播对书籍内容的情感渲染，如对书籍的描述和情感投入，例如对《夏日走过山间》的描述，主播通过语言让书籍内容生动起来，给予观众情感上的共鸣。

信息支持
信息支持涉及提供信息、知识或建议，帮助个体更好地理解某个主题或问题。在图书直播中，信息支持可能包括：
主播对书籍内容的详细介绍，如“这本书它是以这，它是这个自然的启蒙书，里面用了非常多的比喻拟人的方式，然后去记述大自然当中的小动物、植物、昆虫这些。”
提供书籍的使用价值和阅读建议，如“那小朋友也可以从这本书的阅读过程当中感受自然界其实离我们还蛮近的，也非常亲切形象。”
分享作者的背景和书籍的影响力，如对《希达多》作者黑塞的介绍，以及书籍对人生成长的启示。

有形支持
有形支持指的是提供实际帮助或资源，如金钱、物品或服务。在图书直播中，有形支持可能包括：
提供优惠价格和购买链接，如“那这本书今天直播间的到手价格是 10 元。”
主播展示书籍实物，让观众更直观地了解书籍的外观和质感。
提供购买指南和售后服务信息，如库存信息、购买流程、退换货政策等。
通过上述方式，图书直播不仅提供了一个购买书籍的平台，还通过情感交流、信息分享和有形资源的提供，为观众提供了全面的社会支持。
你是文本分类专家，请严格根据以下规则对输入的文本进行分类：
A 情感支持
B 信息支持
C 有形支持
D 以上都不是
注意：
1. 请仅返回 [A/B/C/D]，不要解释原因或生成任何额外内容。
2. 不要创造新的分类。
3. 如果不清楚该文本的分类，请选择 D。
"""

# 调用 prompt 函数，并确保 s0 只输入一次
history = [{"role": "system", "content": ""}]# 初始为空，稍后填写 s0



# 加载数据
input_file = "/social_support.xlsx"
output_file = "/model_name.xlsx"

df = pd.read_excel(input_file)
sentences = df['句子']
results = []

template = """
text:{}
label:
"""

batch_size = 100  # 每批处理200个句子
delay_between_batches = 60  # 每批次处理间隔10秒

# 遍历sentences并调用prompt函数
for i in range(0, len(sentences), batch_size):
    batch_sentences = sentences[i:i+batch_size]
    
    for sentence in batch_sentences:
        s = template.format(sentence)  # 格式化每个句子
        
        # 调用 prompt 函数
        history = prompt(s, history)
        result = history[-1]['content']
        
        # 添加结果到 results 列表
        results.append({'original': sentence, 'result': result})
        
        # 每个请求间的延迟
        time.sleep(0.2)
    
    # 每批次处理后，等待一段时间
    print(f"批次 {i // batch_size + 1} 处理完成，暂停 {delay_between_batches} 秒...")
    time.sleep(delay_between_batches)

# 将结果列表转换为 DataFrame
results_df = pd.DataFrame(results)

# 保存结果到新的xlsx文件
results_df.to_excel(output_file, index=False)

print(f"处理完成，结果已保存至 {output_file}")

In [ ]:
import pandas as pd

# 读取原始文件
input_file = "/model_name.xlsx"
output_file = "/model_name.xlsx"

# 读取Excel文件
df = pd.read_excel(input_file)

# 1. 将 'original' 列名转化为 '句子'
df.rename(columns={'original': '句子'}, inplace=True)

# 2. 将 'result' 列中的标签转换为相应的社会支持类型
def label_to_support(label):
    if label == 'A':
        return '情感支持'
    elif label == 'B':
        return '信息支持'
    elif label == 'C':
        return '有形支持'
    elif label == 'D':
        return '废话'
    else:
        return label  # 如果标签不是A、B、C、D，返回原标签

# 应用转换函数
df['result'] = df['result'].apply(label_to_support)

# 2. 保存转换后的文件
df.to_excel(output_file, index=False)

print(f"转换完成，结果已保存至 {output_file}")

In [ ]:

import pandas as pd 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# 1. 读取数据
truth_file = "/test.xlsx"
pred_file = "/model_name.xlsx"

truth_df = pd.read_excel(truth_file)
pred_df = pd.read_excel(pred_file)

true_labels = truth_df['result']
pred_labels = pred_df['result']

# 2. 计算总体准确率
accuracy = accuracy_score(true_labels, pred_labels)

# 3. 计算 Precision, Recall, F1
categories = ['信息支持', '情感支持', '有形支持', '废话']
precision, recall, f1, support = precision_recall_fscore_support(true_labels, pred_labels, labels=categories, average=None)

# 4. 计算类别级别的准确率（Per-Class Accuracy）
conf_matrix = confusion_matrix(true_labels, pred_labels, labels=categories)
class_accuracies = {}

for i, category in enumerate(categories):
    TP = conf_matrix[i, i]  # 预测正确的样本数
    FN = conf_matrix[i].sum() - TP  # 该类别被错误分类的样本数
    FP = conf_matrix[:, i].sum() - TP  # 其他类别被错误预测为该类别的样本数
    if (TP + FN + FP) > 0:
        class_accuracies[category] = TP / (TP + FN + FP)  # 类内准确率
    else:
        class_accuracies[category] = 0.0  # 防止除零错误

# 5. 计算宏 F1 和加权 F1
macro_f1 = precision_recall_fscore_support(true_labels, pred_labels, labels=categories, average='macro')[2]
weighted_f1 = precision_recall_fscore_support(true_labels, pred_labels, labels=categories, average='weighted')[2]

# 6. 计算加权精确率、未加权平均精确率、加权召回率、未加权平均召回率
precision_weighted = precision_recall_fscore_support(true_labels, pred_labels, labels=categories, average='weighted')[0] * 100
precision_macro = precision_recall_fscore_support(true_labels, pred_labels, labels=categories, average='macro')[0] * 100
recall_weighted = precision_recall_fscore_support(true_labels, pred_labels, labels=categories, average='weighted')[1] * 100
recall_macro = precision_recall_fscore_support(true_labels, pred_labels, labels=categories, average='macro')[1] * 100

# 7. 打印结果
print("分类报告:")
print(f"总体准确率: {accuracy:.4f}")
print(f"{'类别':<12}{'精确率':<12}{'召回率':<12}{'F1分数':<12}{'支持度':<12}{'类别准确率':<12}")
for i, category in enumerate(categories):
    print(f"{category:<12}{precision[i]:<12.4f}{recall[i]:<12.4f}{f1[i]:<12.4f}{support[i]:<12}{class_accuracies[category]:<12.4f}")

print(f"\n宏观F1: {macro_f1:.4f}")
print(f"加权F1: {weighted_f1:.4f}")

# 输出加权精确率、未加权平均精确率、加权召回率、未加权平均召回率
print(f"\n加权精确率: {precision_weighted:.2f}%")
print(f"未加权平均精确率: {precision_macro:.2f}%")
print(f"加权召回率: {recall_weighted:.2f}%")
print(f"未加权平均召回率: {recall_macro:.2f}%")
print(f"加权F1: {weighted_f1 * 100:.2f}%")
print(f"宏F1: {macro_f1 * 100:.2f}%")

